# Entrenamiento de audio con corrupciones
Hecho por:
* Ammi Beltrán
* Fernanda Borja
* Luciano Vidal

**Dirección del dataset**

In [1]:
DATA = "C:\\audio_dataset\\UrbanSound8K"

## Librerías

In [2]:
# Misc
import os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
# Locals
import audio.audio_corrupts as corrupt
import audio.audio_model as audiotrain
import audio.audio_data as dataudio
# Torch
import torch
import torchaudio
from torch import nn
from torch.utils.data import DataLoader

## Dataset

In [3]:
np.random.seed(0)
file = os.path.join(DATA, "metadata", "UrbanSound8K.csv")
df = pd.read_csv(file)
df['relative_path'] = '\\audio' + '\\fold' + df['fold'].astype(str) + '\\' + df['slice_file_name'].astype(str)
df['corrupt'] = np.random.rand(len(df))
df = df[['relative_path', 'classID', 'corrupt']]
df.head()

,relative_path,classID,corrupt
0,\audio\fold5\100032-3-0-0.wav,3,0.548814
1,\audio\fold5\100263-2-0-117.wav,2,0.715189
2,\audio\fold5\100263-2-0-121.wav,2,0.602763
3,\audio\fold5\100263-2-0-126.wav,2,0.544883
4,\audio\fold5\100263-2-0-137.wav,2,0.423655


In [4]:
# from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
# num_items = len(df)
# num_train = round(num_items * 0.8)
# num_val = num_items - num_train
train, val = train_test_split(df, test_size=0.2,random_state=420)
train, val = train.reset_index(drop = True), val.reset_index(drop = True) 
# generator1 = torch.Generator().manual_seed(420)
# train, val = random_split(df, [num_train, num_val], generator=generator1)
print(val.head())
train_ds = dataudio.SoundDS(train, DATA, transform = False) #!!!!
val_ds = dataudio.SoundDS(val, DATA, transform = False)
# print(len(train_ds))
# print(val_ds[1])
#
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=250, shuffle=True, num_workers = 2)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=250, shuffle=False, num_workers = 2)

                     relative_path  classID   corrupt
0     \audio\fold5\17578-5-0-2.wav        5  0.646628
1  \audio\fold1\193698-2-0-114.wav        2  0.479139
2     \audio\fold6\66623-4-0-2.wav        4  0.367986
3      \audio\fold1\9031-3-2-0.wav        3  0.048076
4    \audio\fold7\168846-5-1-4.wav        5  0.095380


In [5]:
model = audiotrain.AudioModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
train = audiotrain.train(model, 150, train_dl, val_dl, criterion, optimizer, state = "test\clean\clean_100.pt", name = "clean", dir = "test/clean", device = "cuda")

Epoch nro 101/150
Iter: 1/28, Loss:0.13300704956054688
Iter: 2/28, Loss:2.822341203689575
Iter: 3/28, Loss:3.1455299854278564
Iter: 4/28, Loss:4.178592681884766
Iter: 5/28, Loss:3.0321547985076904
Iter: 6/28, Loss:1.8448631763458252
Iter: 7/28, Loss:2.3533823490142822
Iter: 8/28, Loss:2.7475388050079346
Iter: 9/28, Loss:2.462961196899414
Iter: 10/28, Loss:3.650273561477661
Iter: 11/28, Loss:4.761714935302734
Iter: 12/28, Loss:3.2320704460144043
Iter: 13/28, Loss:3.337678909301758
Iter: 14/28, Loss:2.411999225616455
Iter: 15/28, Loss:2.021141290664673
Iter: 16/28, Loss:2.1993608474731445
Iter: 17/28, Loss:2.398865222930908
Iter: 18/28, Loss:1.6297715902328491
Iter: 19/28, Loss:1.6867167949676514
Iter: 20/28, Loss:2.090545415878296
Iter: 21/28, Loss:1.4324543476104736
Iter: 22/28, Loss:1.7448947429656982
Iter: 23/28, Loss:2.166134834289551
Iter: 24/28, Loss:1.334059715270996
Iter: 25/28, Loss:1.4327728748321533
Iter: 26/28, Loss:1.4429752826690674
Iter: 27/28, Loss:1.240305781364441
Iter